## Movielens 영화 추천 실습

In [121]:
# MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.
# 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

# 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
# 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
# 별점을 시청횟수로 해석해서 생각하겠습니다.
# 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [122]:
# Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [123]:
import pandas as pd
import os
data_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
data_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv(data_file_path, sep='::', names=data_cols, engine='python')
orginal_data_size = len(data)

In [124]:
# 필요없는 항목은 삭제한다.
data.drop('timestamp', axis=1, inplace=True)

In [125]:
data.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [126]:
# 3점 이상만 남깁니다.
data = data[data['rating']>=3]
filtered_data_size = len(data)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [127]:
# data 컬럼의 이름을 count로 바꿉니다.
data.rename(columns={'rating':'count'}, inplace=True)

In [128]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
# 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 
# MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 2) 분석해 봅시다.

In [130]:
# ratings에 있는 유니크한 영화 개수
# pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.
data['movie_id'].nunique() # 영화갯수

3628

In [131]:
# data에 있는 유니크한 사용자 수
data['user_id'].nunique()

6039

In [132]:
# 가장 인기있는 영화 30개(인기순)
# 영화제목과 같이 보여주기 위해서 movies에서 영화제목을 가져옴
# data1 = data.copy()
data = pd.merge(data, movies[['movie_id', 'title']], how='left', on='movie_id')

In [133]:
data.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [134]:
# 가장 많이 본 영화 순서대로 조회해 보기
data.groupby(['movie_id', 'title'])['user_id'].count().sort_values(ascending=False)[:30]

movie_id  title                                                
2858      American Beauty (1999)                                   3211
260       Star Wars: Episode IV - A New Hope (1977)                2910
1196      Star Wars: Episode V - The Empire Strikes Back (1980)    2885
1210      Star Wars: Episode VI - Return of the Jedi (1983)        2716
2028      Saving Private Ryan (1998)                               2561
589       Terminator 2: Judgment Day (1991)                        2509
593       Silence of the Lambs, The (1991)                         2498
1198      Raiders of the Lost Ark (1981)                           2473
1270      Back to the Future (1985)                                2460
2571      Matrix, The (1999)                                       2434
480       Jurassic Park (1993)                                     2413
2762      Sixth Sense, The (1999)                                  2385
608       Fargo (1996)                                             2371


In [139]:
# 가장 영화를 많이 본 유저 30명(시청 건수가 높은 순)
top_user_count = data.groupby('user_id')['movie_id'].count()
top_user_count.sort_values(ascending=False).head(30)

user_id
4169    1968
4277    1715
1680    1515
3618    1146
1015    1145
5831    1136
2909    1119
1941    1116
424     1106
1980    1054
3224    1034
1285    1034
3539    1029
3391    1012
3032    1003
1181     981
3841     976
4344     973
4448     970
1088     966
549      956
889      943
4725     936
678      933
1448     930
2063     906
3292     897
1698     883
4808     868
2116     855
Name: movie_id, dtype: int64

In [75]:
# 유저별 몇 개의 영화를 보는지에 대한 통계
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [76]:
# 유저별 영화 시청 횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

### 내 데이터로 테스트해보기

In [145]:
# 내가 좋아하는 영화 제목 넣어보기, 
my_favorite = [ 'Fargo (1996)', 'Toy Story (1995)' ,'Braveheart (1995)' ,'Sixth Sense, The (1999)' ,'Pulp Fiction (1994)']

# 영화제목을 movie_id로 매핑하기
movie_title = dict(zip(data['title'], data['movie_id']))
movie_ids = [movie_title[i] for i in my_favorite]
movie_ids

# 6041이라는 user_id가 위 영화를 5회씩 시청했다고 입력하기
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': movie_ids , 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'yesica'이라는 데이터가 없다면
    data = data.append(my_playlist, ignore_index=True)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
else: 
    print('user_id exists!')
data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

user_id exists!


,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,6041,608,5,Fargo (1996)
836479,6041,1,5,Toy Story (1995)
836480,6041,110,5,Braveheart (1995)
836481,6041,2762,5,"Sixth Sense, The (1999)"
836482,6041,296,5,Pulp Fiction (1994)


In [146]:
data['movie_id'].nunique() # 영화갯수는 그래도인지 확인한다

3628

In [147]:
# data에 있는 유니크한 사용자 수가 늘었는지 확인한다
data['user_id'].nunique()  

6040

### 모델에 활용하기 위한 전처리 (실습)¶

In [148]:
# pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

data

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836478,6041,608,5,Fargo (1996)
836479,6041,1,5,Toy Story (1995)
836480,6041,110,5,Braveheart (1995)
836481,6041,2762,5,"Sixth Sense, The (1999)"


In [151]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 한다.
print(movie_to_idx[608])

6039
51


In [152]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie_id column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

# 위의 인덱싱 작업을 통해서 내가 입력한 user_id가 6041에서 6039로 변경된 것을 볼 수 있다. 앞에 2개가 비어있었던 것임을 알 수 있다.
data

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,title
0,0,0,5,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,James and the Giant Peach (1996)
2,0,2,3,My Fair Lady (1964)
3,0,3,4,Erin Brockovich (2000)
4,0,4,5,"Bug's Life, A (1998)"
...,...,...,...,...
836478,6039,51,5,Fargo (1996)
836479,6039,40,5,Toy Story (1995)
836480,6039,87,5,Braveheart (1995)
836481,6039,38,5,"Sixth Sense, The (1999)"


In [161]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 한다.
print(movie_to_idx[608])

6039
51


### 사용자의 명시적/암묵적 평가

In [153]:
# 우리의 데이터셋에서 1회만 영화를 시청한 데이터의 비율은 어떻게 될까요?
# 1회만 영화를 본 데이터의 비율을 보는 코드
only_one = data[data['count']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


## 4) CSR matrix를 직접 만들어 봅시다.

In [154]:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

num_user, num_movie

(6040, 3628)

In [155]:
csr_data = csr_matrix((data['count'].values, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [156]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [157]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [158]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [159]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [160]:
# 모델 학습이 끝났습니다!
# 모델이 1)저의 벡터와 Fargo의 벡터를 어떻게 만들고 있는지와 
# 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다.

In [162]:
yesica, Fargo = user_to_idx[6041], movie_to_idx[608]
yesica_vector, Fargo_vector = als_model.user_factors[yesica], als_model.item_factors[Fargo]

In [163]:
yesica_vector

array([-4.3122342e-01,  6.8834752e-01, -2.9401642e-01, -5.6669271e-01,
       -2.9838184e-01, -1.8841878e-01, -2.8782934e-01,  1.8391201e-01,
        9.7605383e-01,  2.2906011e-02,  3.6476892e-01,  5.4850811e-01,
        1.9270638e-01, -5.2224439e-01, -1.1064228e+00,  7.2050494e-01,
        6.2243718e-01, -1.2952242e+00, -9.2537057e-01,  8.3902538e-01,
        2.1822889e-01,  6.5424077e-02, -3.1604365e-02,  5.0148374e-01,
       -3.6398500e-01,  2.7082229e-02,  2.5084627e-01, -1.2300833e-01,
        5.9881473e-01,  7.6480813e-02, -9.0759415e-03, -2.5834846e-01,
       -9.3152083e-04, -6.9303684e-02, -3.9972574e-01,  1.0817990e-01,
        3.0664524e-01,  1.2876631e+00,  2.5054643e-01,  8.1670445e-01,
        9.3450814e-02, -4.5110904e-02, -1.9275323e-01,  7.0163012e-02,
       -2.6767412e-01,  3.2711804e-01,  7.5801080e-01,  1.5009610e-01,
        1.9278647e-01, -4.1129401e-01,  2.9458505e-01, -3.5582706e-01,
        1.5407865e-01, -2.5100699e-01, -6.2470418e-01,  6.6649085e-01,
      

In [164]:
Fargo_vector

array([ 0.00829652,  0.02620254, -0.01164561, -0.01793665,  0.00592505,
       -0.02559327, -0.00478452,  0.01104585,  0.04097926, -0.00542196,
       -0.00267559, -0.00410714,  0.01853086, -0.01458247, -0.0370547 ,
        0.03358869,  0.0187279 , -0.02361433, -0.0335035 ,  0.03919092,
        0.0189187 ,  0.02238485, -0.00924813,  0.01693362,  0.01304188,
       -0.00625487,  0.00823405,  0.01455276,  0.01340994,  0.00306173,
        0.00530677, -0.02120597, -0.00490639,  0.00322184,  0.00365191,
        0.0026564 ,  0.02050537,  0.04407135,  0.00510569,  0.00519247,
        0.00753709, -0.00191253,  0.00410525,  0.00115213, -0.02691551,
        0.01337338,  0.01756147,  0.00149386, -0.00134249,  0.00375926,
        0.01736114, -0.00829673,  0.0185973 , -0.00305783,  0.01331178,
        0.02032025, -0.01090456,  0.01702393, -0.04376386,  0.04903769,
       -0.01238407,  0.02103529,  0.00043712, -0.00152117,  0.00939264,
        0.01644825,  0.01210486,  0.00901257, -0.01254624,  0.00

In [165]:
# yesica와 Fargo를 내적하는 코드
np.dot(yesica_vector, Fargo_vector)

0.6318173

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [166]:
# 모델이 yesica의 Braveheart에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다
Braveheart = movie_to_idx[110]
Braveheart_vector = als_model.item_factors[Braveheart]
np.dot(yesica_vector, Braveheart_vector)

0.50156343

In [167]:
favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.19740087),
 (50, 0.15664709),
 (33, 0.11311597),
 (4, 0.106270224),
 (322, 0.10599119),
 (110, 0.10328289),
 (330, 0.09223877),
 (10, 0.08601447),
 (20, 0.08022782),
 (255, 0.07780574),
 (34, 0.07535057),
 (126, 0.07189051),
 (32, 0.07085998),
 (16, 0.068429574),
 (478, 0.06506795)]

In [168]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 588,
 2355,
 34,
 1265,
 364,
 595,
 2321,
 1923,
 1907,
 2396,
 1566,
 2687,
 3253]

In [169]:
favorite_movie = 1  # 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.19740087),
 (50, 0.15664709),
 (33, 0.11311597),
 (4, 0.106270224),
 (322, 0.10599119),
 (110, 0.10328289),
 (330, 0.09223877),
 (10, 0.08601447),
 (20, 0.08022782),
 (255, 0.07780574),
 (34, 0.07535057),
 (126, 0.07189051),
 (32, 0.07085998),
 (16, 0.068429574),
 (478, 0.06506795)]

In [170]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 588,
 2355,
 34,
 1265,
 364,
 595,
 2321,
 1923,
 1907,
 2396,
 1566,
 2687,
 3253]

In [171]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [172]:
get_similar_movie(40)

[40, 2341, 1650, 767, 1624, 83, 3374, 1725, 1044, 1728]

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [173]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(121, 0.64571655),
 (157, 0.5462611),
 (99, 0.49720982),
 (233, 0.46493375),
 (23, 0.44185054),
 (48, 0.41760603),
 (269, 0.40214303),
 (224, 0.39909887),
 (50, 0.3937905),
 (322, 0.3692599),
 (248, 0.33925518),
 (126, 0.32951292),
 (92, 0.3155436),
 (4, 0.2937866),
 (472, 0.2835997),
 (141, 0.28104275),
 (220, 0.27112475),
 (170, 0.26997226),
 (289, 0.26561153),
 (110, 0.2617551)]

In [174]:
[idx_to_movie[i[0]] for i in movie_recommended]

[593,
 318,
 2858,
 50,
 527,
 2028,
 1213,
 1617,
 3114,
 34,
 1704,
 2396,
 589,
 2355,
 1358,
 457,
 47,
 2997,
 1089,
 1265]

In [175]:
Toy_Story = movie_to_idx[1]
explain = als_model.explain(user, csr_data, itemid=Toy_Story)

In [176]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1, 0.43321928227072176),
 (2762, 0.04156296473189755),
 (608, 0.0003147121965448008),
 (110, -0.012440289922506552),
 (296, -0.015847483011066223)]